In [6]:
import numpy as np 
import pandas as pd 
# from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.utils import pad_sequences

In [9]:
data = pd.read_csv(r"/kaggle/input/first-gop-debate-twitter-sentiment/Sentiment.csv")

In [34]:
data = data[['text', 'sentiment']]
data.head()

,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


In [35]:
data = data[data['sentiment']!="Neutral"]
data.head()

,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


In [36]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z\s0-9]','',x)))

In [43]:
for idx,row in data.iterrows():
    row[0] = row[0].replace("rt"," ")

In [46]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
x = tokenizer.texts_to_sequences(data['text'].values)
x = pad_sequences(x)

In [65]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = x.shape[1]))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(lstm_out, dropout=0.4))

model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [66]:
y = pd.get_dummies(data['sentiment']).values
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 11)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(7510, 28) (7510, 2)
(3219, 28) (3219, 2)


In [67]:
model.fit(X_train, y_train,
          epochs = 15,
          batch_size=64,
          verbose = 2,
        validation_data=(X_test,y_test))

Epoch 1/15
118/118 - 12s - loss: 0.4768 - accuracy: 0.7989 - val_loss: 0.3895 - val_accuracy: 0.8254 - 12s/epoch - 103ms/step
Epoch 2/15
118/118 - 9s - loss: 0.3819 - accuracy: 0.8390 - val_loss: 0.3549 - val_accuracy: 0.8515 - 9s/epoch - 80ms/step
Epoch 3/15
118/118 - 9s - loss: 0.3324 - accuracy: 0.8609 - val_loss: 0.3461 - val_accuracy: 0.8618 - 9s/epoch - 77ms/step
Epoch 4/15
118/118 - 9s - loss: 0.3059 - accuracy: 0.8711 - val_loss: 0.3632 - val_accuracy: 0.8596 - 9s/epoch - 77ms/step
Epoch 5/15
118/118 - 10s - loss: 0.2957 - accuracy: 0.8758 - val_loss: 0.3374 - val_accuracy: 0.8646 - 10s/epoch - 82ms/step
Epoch 6/15
118/118 - 9s - loss: 0.2757 - accuracy: 0.8834 - val_loss: 0.3380 - val_accuracy: 0.8605 - 9s/epoch - 77ms/step
Epoch 7/15
118/118 - 9s - loss: 0.2583 - accuracy: 0.8905 - val_loss: 0.3523 - val_accuracy: 0.8621 - 9s/epoch - 77ms/step
Epoch 8/15
118/118 - 9s - loss: 0.2548 - accuracy: 0.8951 - val_loss: 0.3413 - val_accuracy: 0.8658 - 9s/epoch - 76ms/step
Epoch 9/15


In [68]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = 64)

print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

51/51 - 1s - loss: 0.4300 - accuracy: 0.8571 - 1s/epoch - 24ms/step
score: 0.43
acc: 0.86
